# 🧠 Modèle MMM Complet - Adstock & Saturation

Ce notebook implémente le modèle MMM bayésien complet avec :
1. **Modèle simple** : Régression linéaire bayésienne
2. **+ Adstock** : Persistance temporelle de l'effet pub
3. **+ Saturation** : Rendements décroissants
4. **Comparaison** : Quel modèle explique le mieux les données ?

**Auteur** : Ivan  
**Projet** : MMM Bayésien - MSMIN5IN43

In [ ]:
# Imports
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from data.loader import load_csv_data, split_train_test
from data.preprocessing import prepare_mmm_data
from models.base_mmm import BayesianMMM
from inference.diagnostics import check_convergence

# Config
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
%matplotlib inline

# Seed pour reproductibilité
np.random.seed(42)

## 1️⃣ Chargement et préparation des données

In [ ]:
# Charger les données
df = load_csv_data('../data/raw/sample_data.csv')

print(f"✓ Dataset chargé : {len(df)} périodes")
print(f"✓ Colonnes : {list(df.columns)}")
print(f"\n📅 Période : {df['date'].min()} → {df['date'].max()}")

df.head()

In [ ]:
# Split train/test (80/20)
train_df, test_df = split_train_test(df, train_ratio=0.8)

print(f"Train : {len(train_df)} périodes")
print(f"Test  : {len(test_df)} périodes")

In [ ]:
# Préparer les données
train_prep, meta = prepare_mmm_data(train_df, normalize=True)
test_prep, _ = prepare_mmm_data(test_df, normalize=True)

# Extraire X et y
media_cols = ['media_1_spend', 'media_2_spend', 'media_3_spend']

X_train = train_prep[media_cols].values
y_train = train_prep['sales_log'].values

X_test = test_prep[media_cols].values
y_test = test_prep['sales_log'].values

print(f"✓ X_train : {X_train.shape}")
print(f"✓ y_train : {y_train.shape}")
print(f"✓ X_test  : {X_test.shape}")
print(f"✓ y_test  : {y_test.shape}")

## 2️⃣ Modèle 1 : Simple (sans transformations)

**Équation** : `sales = β₀ + β₁·media₁ + β₂·media₂ + β₃·media₃ + ε`

In [ ]:
# Construire et entraîner
print("🔨 Construction du modèle simple...")
mmm_simple = BayesianMMM(use_adstock=False, use_saturation=False)

print("⏳ Sampling MCMC (cela prend ~2-3 minutes)...")
trace_simple = mmm_simple.fit(
    X_train, y_train,
    draws=1000,
    tune=1000,
    chains=2,
    random_seed=42
)

print("\n✅ Modèle simple entraîné !")

In [ ]:
# Diagnostics de convergence
report_simple = check_convergence(trace_simple)

print("📊 Diagnostics de convergence\n")
print(f"Convergence : {'✅ OUI' if report_simple['converged'] else '❌ NON'}")
print(f"R-hat max   : {report_simple['r_hat_max']:.4f}")
print(f"ESS bulk min: {report_simple['ess_bulk_min']:.0f}")
print(f"Divergences : {report_simple['n_divergences']}")

if report_simple['warnings']:
    print("\n⚠️  Warnings:")
    for w in report_simple['warnings']:
        print(f"  - {w}")

In [ ]:
# Résumé des paramètres
print("📈 Paramètres estimés (modèle simple)\n")
summary_simple = mmm_simple.summary()
print(summary_simple[['mean', 'sd', 'hdi_3%', 'hdi_97%']].round(3))

In [ ]:
# Prédictions
y_pred_simple = mmm_simple.predict(X_test)

mae_simple = np.mean(np.abs(y_test - y_pred_simple))
rmse_simple = np.sqrt(np.mean((y_test - y_pred_simple) ** 2))

print(f"📊 Performance sur test set (modèle simple)")
print(f"MAE  : {mae_simple:.4f}")
print(f"RMSE : {rmse_simple:.4f}")

## 3️⃣ Modèle 2 : Avec Adstock

**Ajout** : Effet persistant de la publicité

**Transformation** : `x_adstocked[t] = x[t] + α·x_adstocked[t-1]`

In [ ]:
# Construire et entraîner
print("🔨 Construction du modèle avec adstock...")
mmm_adstock = BayesianMMM(use_adstock=True, use_saturation=False)

print("⏳ Sampling MCMC (cela prend ~3-4 minutes)...")
trace_adstock = mmm_adstock.fit(
    X_train, y_train,
    draws=1000,
    tune=1000,
    chains=2,
    random_seed=42
)

print("\n✅ Modèle adstock entraîné !")

In [ ]:
# Diagnostics
report_adstock = check_convergence(trace_adstock)

print("📊 Diagnostics (modèle adstock)\n")
print(f"Convergence : {'✅ OUI' if report_adstock['converged'] else '❌ NON'}")
print(f"R-hat max   : {report_adstock['r_hat_max']:.4f}")
print(f"ESS bulk min: {report_adstock['ess_bulk_min']:.0f}")

In [ ]:
# Paramètres alpha (taux de rétention)
alpha_mean = trace_adstock.posterior['alpha'].mean(dim=['chain', 'draw']).values

print("📈 Taux de rétention (alpha) par canal\n")
for i, alpha in enumerate(alpha_mean):
    print(f"Canal {i+1} : α = {alpha:.3f} → effet persiste ~{int(1/(1-alpha))} périodes")

In [ ]:
# Performance
y_pred_adstock = mmm_adstock.predict(X_test)

mae_adstock = np.mean(np.abs(y_test - y_pred_adstock))
rmse_adstock = np.sqrt(np.mean((y_test - y_pred_adstock) ** 2))

print(f"📊 Performance sur test set (modèle adstock)")
print(f"MAE  : {mae_adstock:.4f}")
print(f"RMSE : {rmse_adstock:.4f}")

## 4️⃣ Modèle 3 : Complet (Adstock + Saturation)

**Ajout** : Rendements décroissants

**Transformation** : `x_saturated = x^s / (k^s + x^s)`

In [ ]:
# Construire et entraîner
print("🔨 Construction du modèle complet...")
mmm_full = BayesianMMM(use_adstock=True, use_saturation=True)

print("⏳ Sampling MCMC (cela prend ~4-5 minutes)...")
trace_full = mmm_full.fit(
    X_train, y_train,
    draws=1000,
    tune=1000,
    chains=2,
    random_seed=42
)

print("\n✅ Modèle complet entraîné !")

In [ ]:
# Diagnostics
report_full = check_convergence(trace_full)

print("📊 Diagnostics (modèle complet)\n")
print(f"Convergence : {'✅ OUI' if report_full['converged'] else '❌ NON'}")
print(f"R-hat max   : {report_full['r_hat_max']:.4f}")
print(f"ESS bulk min: {report_full['ess_bulk_min']:.0f}")

In [ ]:
# Paramètres de saturation
k_mean = trace_full.posterior['half_saturation'].mean(dim=['chain', 'draw']).values
s_mean = trace_full.posterior['slope'].mean(dim=['chain', 'draw']).values

print("📈 Paramètres de saturation\n")
for i in range(len(k_mean)):
    print(f"Canal {i+1} : k={k_mean[i]:.2f}, s={s_mean[i]:.2f}")

In [ ]:
# Performance
y_pred_full = mmm_full.predict(X_test)

mae_full = np.mean(np.abs(y_test - y_pred_full))
rmse_full = np.sqrt(np.mean((y_test - y_pred_full) ** 2))

print(f"📊 Performance sur test set (modèle complet)")
print(f"MAE  : {mae_full:.4f}")
print(f"RMSE : {rmse_full:.4f}")

## 5️⃣ Comparaison des modèles

In [ ]:
# Tableau comparatif
comparison = pd.DataFrame({
    'Modèle': ['Simple', 'Adstock', 'Complet'],
    'MAE': [mae_simple, mae_adstock, mae_full],
    'RMSE': [rmse_simple, rmse_adstock, rmse_full],
    'Convergence': [
        '✅' if report_simple['converged'] else '❌',
        '✅' if report_adstock['converged'] else '❌',
        '✅' if report_full['converged'] else '❌'
    ]
})

print("\n📊 COMPARAISON DES MODÈLES\n")
print(comparison.to_string(index=False))

# Meilleur modèle
best_idx = comparison['MAE'].idxmin()
print(f"\n🏆 Meilleur modèle : {comparison.loc[best_idx, 'Modèle']}")

In [ ]:
# Visualisation des prédictions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = [
    ('Simple', y_pred_simple, mae_simple),
    ('Adstock', y_pred_adstock, mae_adstock),
    ('Complet', y_pred_full, mae_full)
]

for ax, (name, y_pred, mae) in zip(axes, models):
    ax.scatter(y_test, y_pred, alpha=0.6)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    ax.set_xlabel('Ventes réelles (log)', fontsize=11)
    ax.set_ylabel('Ventes prédites (log)', fontsize=11)
    ax.set_title(f'{name}\nMAE: {mae:.4f}', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6️⃣ Analyse du modèle complet

In [ ]:
# Trace plots (convergence visuelle)
az.plot_trace(trace_full, var_names=['beta_media', 'alpha', 'half_saturation'])
plt.tight_layout()
plt.show()

In [ ]:
# Distributions a posteriori
az.plot_posterior(trace_full, var_names=['beta_media', 'alpha'])
plt.tight_layout()
plt.show()

In [ ]:
# Contributions par canal
contributions = mmm_full.get_channel_contributions()
contributions['pct'] = (contributions['total_contribution'] / 
                        contributions['total_contribution'].sum() * 100)

print("\n💰 CONTRIBUTIONS PAR CANAL\n")
print(contributions.round(2))

# Visualisation
plt.figure(figsize=(10, 6))
plt.bar(contributions['channel'], contributions['pct'])
plt.xlabel('Canal', fontsize=12)
plt.ylabel('Contribution (%)', fontsize=12)
plt.title('Contribution de chaque canal aux ventes', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 7️⃣ Conclusions

### ✅ Résultats clés

**Performance :**
- Modèle simple : baseline
- Modèle adstock : capture la persistance temporelle
- Modèle complet : capture adstock + saturation

**Insights business :**
1. **Adstock** : Effet pub persiste X semaines
2. **Saturation** : Rendements décroissants après Y€ de dépenses
3. **Attribution** : Contributions relatives par canal

**Prochaines étapes :**
1. Optimisation budgétaire
2. Scénarios what-if
3. Validation croisée
4. Déploiement